In [1]:
using AdaptiveHierarchicalRegularBinning
using LinearAlgebra
using MLDatasets
using BenchmarkTools

In [2]:
X = MNIST(split=:train).features
X = reshape(X, prod(size(X)[1:end-1]), :)
X = transpose(X);

In [3]:
U, S, V = svd(X)
Np = 10
Up, Sp, Vp = U[:, 1:Np], S[1:Np], V[1:Np, 1:Np]

Xp = Up * Diagonal(Sp) * transpose(Vp);

In [4]:
V = Xp
R = Vector{UInt}(undef, size(Xp, 1))
l = 6

@benchmark AdaptiveHierarchicalRegularBinning.spatial_encode!(R, V, $l; dims=Val(1), center=false)

BenchmarkTools.Trial: 1259 samples with 1 evaluation.
 Range (min … max):  3.943 ms …   9.188 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.949 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.970 ms ± 227.023 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁▃█▇▄▃▂▁▁▁                                                   
  ███████████▅▆▆▆▆▄▄▁▄▄▇▄▄▁▁▁▁▁▄▁▁▄▁▄▄▁▄▁▄▄▄▄▁▄▄▄▁▄▁▁▁▁▁▁▄▁▄▄ █
  3.94 ms      Histogram: log(frequency) by time      4.07 ms <

 Memory estimate: 1.56 KiB, allocs estimate: 24.

In [7]:
Va = copy(V)
Ra = copy(R)
Ia = collect(UInt, 1:length(R))

Vb = similar(V)
Rb = similar(R)
Ib = Vector{UInt}(undef, length(R))

P = falses(length(R))

rsd = AdaptiveHierarchicalRegularBinning.RadixSortDetails(8, 1, length(R); dims=1, sml_th=1)

# @benchmark(
#   AdaptiveHierarchicalRegularBinning.radixsort_seq_seq_impl!(Va, Ra, Ia, Vb, Rb, Ib, P, rsd),
#   setup=begin
#     Va .= V
#     Ra .= R
#     Ia .= 1:length(R)

#     P .= false
#   end
# )

@time AdaptiveHierarchicalRegularBinning.radixsort_seq_seq_impl!(Va, Ra, Ia, Vb, Rb, Ib, P, rsd)

In [6]:
Ra[P] .= Rb[P]

issorted(Ra)

false